In [1]:
pip install tensorflow librosa numpy matplotlib jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 31.4 MB/s eta 0:00:00


In [2]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt

In [3]:
def process_audio(audio_path):
    y,sr = librosa.load(audio_path,sr=16000)
    mel_spec = librosa.feature.melspectrogram(y=y,sr=sr,n_mels=128)
    mel_spec_db = librosa.power_to_db(mel_spec,ref=np.max)

    plt.figure(figsize=(10,4))
    librosa.display.specshow(mel_spec_db,sr=sr,x_axis="time",y_axis="mel")
    plt.colorbar(format="%+2.0f dB")
    plt.title("Mel Spectrogram")
    plt.show()

    return mel_spec


In [4]:
# process_audio("/content/ttsMP3.com_VoiceText_2025-3-11_12-8-46.wav")

In [5]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import torch

processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-large-960h")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
def transcribe(audio_path):
    y,sr = librosa.load(audio_path,sr=16000)
    input_values = processor(y,return_tensors='pt',sampling_rate=sr).input_values

    with torch.no_grad():
        logits = model(input_values).logits

    predicted_ids = torch.argmax(logits,dim=-1)
    transcription = processor.batch_decode(predicted_ids)[0]
    return transcription

In [7]:
# print(transcribe('/content/ttsMP3.com_VoiceText_2025-3-11_12-8-46.wav'))

In [8]:
# from transformers import TrainingArguments, Trainer

# training_args = TrainingArguments(
#     output_dir="./wav2vec2_model",
#     per_device_train_batch_size=8,
#     evaluation_strategy="epoch",
#     save_strategy="epoch",
#     learning_rate=3e-5,
#     num_train_epochs=3,
#     logging_dir="./logs"
# )

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,  # Your custom dataset
#     eval_dataset=eval_dataset
# )

# trainer.train()

In [9]:
# print(transcribe('Rec_wav.wav'))

<ipython-input-6-551e27b2caf3>:2: UserWarning: PySoundFile failed. Trying audioread instead.
  y,sr = librosa.load(audio_path,sr=16000)
/usr/local/lib/python3.11/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


FileNotFoundError: [Errno 2] No such file or directory: 'Rec_wav.wav'

In [10]:

import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import jiwer

# Load pre-trained Wav2Vec2 model
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-large-960h")

def process_audio(audio_path):
    y, sr = librosa.load(audio_path, sr=16000)

    # Silence removal
    non_silent_intervals = librosa.effects.split(y, top_db=20)
    y_trimmed = np.concatenate([y[start:end] for start, end in non_silent_intervals])

    # Feature extraction
    mel_spec = librosa.feature.melspectrogram(y=y_trimmed, sr=sr, n_mels=128)
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)

    # Plot Mel Spectrogram
    plt.figure(figsize=(10, 4))
    librosa.display.specshow(mel_spec_db, sr=sr, x_axis="time", y_axis="mel")
    plt.colorbar(format="%+2.0f dB")
    plt.title("Mel Spectrogram (Trimmed Audio)")
    plt.show()

    return y_trimmed, sr

def transcribe(audio_path):
    y, sr = process_audio(audio_path)

    input_values = processor(y, return_tensors="pt", sampling_rate=sr).input_values
    with torch.no_grad():
        logits = model(input_values).logits

    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids)[0]

    return transcription

# Evaluation: Word Error Rate (WER)
def evaluate_transcription(reference_text, transcribed_text):
    wer_score = jiwer.wer(reference_text, transcribed_text)
    print(f"Word Error Rate (WER): {wer_score * 100:.2f}%")
    return wer_score


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
